In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import dipy.reconst.peaks as drp
sns.set(style='whitegrid')

In [29]:
import os.path as op

import numpy as np
import scipy.ndimage as ndi

import nibabel as nib
import dipy.reconst.sfm as sfm
import dipy.core.gradients as grad
import dipy.core.optimize as opt
import dipy.data as dpd
from dipy.direction import DeterministicMaximumDirectionGetter, ProbabilisticDirectionGetter
import dipy.viz.projections as proj
from dipy.tracking.local import LocalTracking,ThresholdTissueClassifier
from dipy.tracking.utils import seeds_from_mask
import dipy.viz.fvtk as fvtk
from dipy.viz.colormap import line_colors
from dipy.tracking.utils import move_streamlines
from numpy.linalg import inv


import scm.scm as scm

In [3]:
hardi_img, gtab, labels_img = dpd.read_stanford_labels()
data = hardi_img.get_data()
affine = hardi_img.get_affine()
#data = data[40:50, 40:50, 40:50]
labels = labels_img.get_data()
mask = (labels == 1) | (labels == 2)

Dataset is already in place. If you want to fetch it again, please first remove the folder /Users/arokem/.dipy/stanford_hardi 
All files already in /Users/arokem/.dipy/stanford_hardi.


In [4]:
n_split = 75

data_train = np.concatenate([data[..., :5], data[..., 10:85]], -1)
data_test = np.concatenate([data[..., 5:10], data[..., 85:]], -1)

gtab_train = grad.gradient_table_from_bvals_bvecs(np.concatenate([gtab.bvals[:5], gtab.bvals[10:85]]), 
                                                  np.concatenate([gtab.bvecs[:5], gtab.bvecs[10:85]]))

gtab_test = grad.gradient_table_from_bvals_bvecs(np.concatenate([gtab.bvals[5:10], gtab.bvals[85:]]), 
                                                 np.concatenate([gtab.bvecs[5:10], gtab.bvecs[85:]]))

In [5]:
scmodel = scm.SpatContModel(gtab_train)
sfmodel = sfm.SparseFascicleModel(gtab_train)

In [6]:
sffit = sfm.SparseFascicleFit(sfmodel, np.load('sf_beta.npy'), np.load('sf_S0.npy'), np.load('sf_mean.npy'))

In [7]:
sc_beta = np.load('sc_beta.npy')

In [8]:
scfit = scm.SpatContFit(scmodel, np.load('sc_beta.npy'), np.load('sc_S0.npy'), np.load('sc_mean.npy'))

In [9]:
scodf = scfit.odf(scmodel.sphere)

In [ ]:
ren = fvtk.ren()
scodf_small = scodf[20:50, 55:85, 38:39]
scodf_spheres = fvtk.sphere_funcs(scodf_small, scmodel.sphere, scale=0.13, norm=False)
fvtk.add(ren, scodf_spheres)
fvtk.record(ren, out_path='sc_odfs.png', size=(600,600))

In [12]:
LV1_ni = nib.load(op.join('/Users/arokem/projects/jov-white-matter-matters/data', 'SUB1_LV1.nii.gz'))
LV1_data = LV1_ni.get_data()
V1 = (LV1_data == 1)
V1_extended = ndi.gaussian_filter(V1.astype(float), sigma=0.25).astype(bool)

In [13]:
seeds = seeds_from_mask(V1_extended, density=[2, 2, 2], affine=affine)
classifier = ThresholdTissueClassifier(mask.astype(float), 0.0)

In [33]:
sl = []
for fit, model in zip([scfit, sffit], [scmodel, sfmodel]):
    #dir_getter = DeterministicMaximumDirectionGetter.from_pmf(fit.odf(model.sphere), 90, sphere=model.sphere)
    dir_getter = ProbabilisticDirectionGetter.from_pmf(fit.odf(model.sphere), 30, sphere=model.sphere)
    streamlines = LocalTracking(dir_getter, classifier, seeds, affine, step_size=.5)
    len_th = 10
    sl.append([s for s in streamlines if s.shape[0]>len_th])

In [34]:
t1 = nib.load(op.join('/Users/arokem/projects/jov-white-matter-matters/data', 'SUB1_t1_resamp.nii.gz'))
t1_data = t1.get_data()
t1_aff = t1.get_affine()
color = line_colors(sl[0])

In [36]:
for this_sl, label in zip(sl, ['spat-cont', 'sfm']):
    ren = fvtk.ren()
    streamlines_actor = fvtk.streamtube(list(move_streamlines(this_sl, inv(t1_aff))),
                                    line_colors(this_sl))
    vol_actor = fvtk.slicer(t1_data, voxsz=(1.0, 1.0, 1.0), plane_i=[40],
                        plane_j=None, plane_k=[25], outline=False)

    
    fvtk.add(ren, streamlines_actor)
    fvtk.add(ren, vol_actor)
    fvtk.camera(ren, viewup=(1,0,1), verbose=False)
    fvtk.record(ren, out_path='%s-prob-track.png'%label, size=(600,600))

In [37]:
import dipy.tracking.life as life
fiber_model = life.FiberModel(gtab)

In [ ]:
fit_sc = fiber_model.fit(data, sl[0], affine=np.linalg.inv(affine))

In [ ]:
fit_sf = fiber_model.fit(data, sl[1], affine=np.linalg.inv(affine))

In [ ]:
rmse = []

for fiber_fit in [fit_sc, fit_sf]:
    model_predict = fiber_fit.predict()
    model_error = model_predict - fiber_fit.data
    model_rmse = np.sqrt(np.mean(model_error[:, 10:] ** 2, -1))
    rmse.append(model_rmse)

In [ ]:
fig, ax = plt.subplots(1)
ax.hist(rmse[0], histtype='step', linewidth=2)
ax.hist(rmse[1], histtype='step', linewidth=2)

In [ ]:
fit_sc.vox_coords.shape

In [ ]:
fit_sf.vox_coords.shape

In [ ]:
np.median(rmse[0])

In [ ]:
np.median(rmse[1])